In [1]:
import os
import sys
from typing import Callable

this_module = sys.modules[__name__]

def get_input() -> list[str]:
    return get_lines_from_file('./input')

def get_test_input(idx) -> list[str]:
    if os.path.isfile(f'./test_input{idx}'):
        return get_lines_from_file(f'./test_input{idx}')
    else:
        return get_lines_from_file('./test_input')
    
def get_test_result(idx) -> int:
    return get_int_from_file(f'./test_result{idx}')

def get_solution_func(idx) -> Callable:
    return getattr(this_module, f'solution{idx}')

def get_lines_from_file(filepath) -> list[str]:
    with open(filepath) as f:
        return [line.strip('\n') for line in f.readlines()]

def get_str_from_file(filepath) -> str:
    with open(filepath) as f:
        return f.readline().strip('\n')

def get_int_from_file(filepath) -> int:
    with open(filepath) as f:
        return int(f.readline().strip())

def log_invocation(func):
    def logged_func(*args):
        res = func(*args)
        print(f'{func.__name__}({args}) -> {res}')
        return res
    return logged_func
    
def run_test(idx: int) -> bool:
    res = get_solution_func(idx)(get_test_input(idx))
    test_res = get_test_result(idx)
    
    if test_res == res:
        print(f'Your solution for part {idx} works!!! :) (on the test input, that is)')
        print(f'Let`s try it on the actual input now...')
        return True
    else:
        print(f'Your solution for part {idx} does not work yet. Keep going!')
        print(f'You`ve got {res}, but the correct test result is {test_res}')
        return False

def run_solution(idx: int):
    sol = get_solution_func(idx)(get_input())
    print(f'The solution for part {idx} is: {sol}')

def run():
    if run_test(1):
        run_solution(1)
        print('\nOn to part 2...\n')
        if run_test(2):
            run_solution(2)

In [21]:
import re
from typing import Any, NamedTuple, Tuple

class Coord(NamedTuple):
    col: int
    row: int

class Number(NamedTuple):
    value: int
    coordinates: list[Coord]

class Symbol(NamedTuple):
    char: str
    coordinate: Coord


def parse_numbers(s: str, row: int) -> list[Number]:
    numbers = []
    if (m := re.finditer(r'\d+', s)) is not None:
        for f in m:
            numbers.append(Number(int(f.group()), generate_coordinates(f.group(), f.start(), row)))
        return numbers
    return None

def parse_symbols(s: str, row: int) -> list[Symbol]:
    symbols = []
    if (m := re.finditer(r'[^\d.]', s)) is not None:
        for f in m:
            symbols.append(Symbol(f.group(), Coord(f.start(), row)))
        return symbols
    return None

def generate_coordinates(s: str, col: int, row: int) -> list[Coord]:
    coords = []
    for i in range(len(s)):
        coords.append(Coord(col+i, row))
    return coords


def parse_schematic(schematic_lines: list[str]) -> Tuple[list[Number], list[Symbol]]:
    numbers = []
    symbols = []

    for row_idx, line in enumerate(schematic_lines):
        if (res := parse_numbers(line, row_idx)) is not None:
            numbers += res
        if (res := parse_symbols(line, row_idx)) is not None:
            symbols += res

    return numbers, symbols

def create_numbers_map(numbers: list[Number]) -> dict[Coord, Number]:
    coord_map = {}
    
    for num in numbers:
        for coord in num.coordinates:
            coord_map[coord] = num

    return coord_map

def unique_list(l: list) -> list:
    unique_list = []
    for x in l:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

def get_adjacent_numbers(numbers_map: dict, symbol: Symbol) -> list[Number]:
    adj_numbers: list[Number] = []

    if (c := Coord(symbol.coordinate.col-1, symbol.coordinate.row)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])
    if (c := Coord(symbol.coordinate.col-1, symbol.coordinate.row-1)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])
    if (c := Coord(symbol.coordinate.col, symbol.coordinate.row-1)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])
    if (c := Coord(symbol.coordinate.col+1, symbol.coordinate.row-1)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])
    if (c := Coord(symbol.coordinate.col+1, symbol.coordinate.row)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])
    if (c := Coord(symbol.coordinate.col+1, symbol.coordinate.row+1)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])
    if (c := Coord(symbol.coordinate.col, symbol.coordinate.row+1)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])
    if (c := Coord(symbol.coordinate.col-1, symbol.coordinate.row+1)) in numbers_map.keys():
        adj_numbers.append(numbers_map[c])

    return unique_list(adj_numbers)


def solution1(input: list[str]) -> int:
    numbers, symbols = parse_schematic(input)
    numbers_map = create_numbers_map(numbers)
    
    part_numbers: list[Number] = []
    for sym in symbols:
        part_numbers += get_adjacent_numbers(numbers_map, sym)

    return sum(map(lambda n: n.value, part_numbers))

def solution2(input: list[str]) -> int:
    # Your code goes here...
    return 0

run()

Your solution for part 1 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 1 is: 536202

On to part 2...

Your solution for part 2 does not work yet. Keep going!
You`ve got 0, but the correct test result is -1
